# Polyglot notebook prototype
Jupyter notebooks are a nice way to try out F# code combined with some Markdown documentation. This notebook is being added to try this out and see if we like it.

Requirements
- Visual Studio Code
- [Polyglot Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) extension for VS code
- .NET 6 SDK

Quick start
- Click into a code cell and press the run button or CTRL + ALT + ENTER to execute
- Double click a markdown cell to edit then click the check mark button to save it
- Hover to get options to add a Code or a Markdown cell
- Click the Language in the lower right of the cell to change it


## Higher order input function
Any `int -> int` function can be passed to the `fn` argument of `evalWith5ThenAdd2`

In [1]:
let add1 x = x + 1
let add2 x = x + 2
let double x = x * 2

let evalWith5ThenAdd2 fn = fn 5 + 2

printfn $"evalWith5ThenAdd2 add1: %d{evalWith5ThenAdd2 add1}"
printfn $"evalWith5ThenAdd2 add2: %d{evalWith5ThenAdd2 add2}"
printfn $"evalWith5ThenAdd2 double: %d{evalWith5ThenAdd2 double}"

evalWith5ThenAdd2 add1: 8
evalWith5ThenAdd2 add2: 9
evalWith5ThenAdd2 double: 12


## Higher order output function
Functions can return functions

In [2]:
let adderGenerator numberToAdd = (+) numberToAdd
let add1 = adderGenerator 1
let add2 = adderGenerator 2

printfn $"add1 2: %d{add1 2}"
printfn $"add2 2: %d{add2 2}"

add1 2: 3
add2 2: 4


## Printing
The `%A` format specifier will print structured data.
[Table of all format specifiers](https://learn.microsoft.com/en-us/dotnet/fsharp/language-reference/plaintext-formatting#format-specifiers-for-printf)

In [11]:
printfn "%A" [ for i in 1 .. 5 -> [ for j in 1 .. 5 -> (i, j) ] ]

[[(1, 1); (1, 2); (1, 3); (1, 4); (1, 5)];
 [(2, 1); (2, 2); (2, 3); (2, 4); (2, 5)];
 [(3, 1); (3, 2); (3, 3); (3, 4); (3, 5)];
 [(4, 1); (4, 2); (4, 3); (4, 4); (4, 5)];
 [(5, 1); (5, 2); (5, 3); (5, 4); (5, 5)]]


## Generic arguments
The `x` in this function has the type `'a` which is the equivalent of `OnAStick<T>` in C#.

In [4]:
let onAStick x = x.ToString() + " on a stick"

let onAStick29 = onAStick 29
let onAStickPi = onAStick 3.14159
let onAStickSg = onAStick "sausage"

printfn "%A" onAStick29
printfn "%A" onAStickPi
printfn "%A" onAStickSg

"29 on a stick"
"3.14159 on a stick"
"sausage on a stick"


## Partial application
Supply less than the full set of arguments to a function and you get back a new function with those values "baked" in. The new function will then take which ever arguments were left off when the original function was called. This is a great way to handle dependencies that do I/O. In the production code there can be function that calls the real database and in the test code there can be functions that return canned values kind of like mocking libraries do.

In [5]:
let adderWithLogger logger x y =
    let result = x + y
    logger "result" result
    result

let jsonLogger argName argValue =
    printfn $"""{{ "logMessage": "{argName}={argValue}" }}"""

let xmlLogger argName argValue =
    printfn $"<logMessage>{argName}={argValue}</logMessage"

// Call the adderWithLogger function and supply only the logger argument to build new 
// functions with each logger embedded.
let adderWithJsonLogger = adderWithLogger jsonLogger
let adderWithXmlLogger = adderWithLogger xmlLogger

// Now call the new functions and supply the remaining arguments...
let sum1 = adderWithJsonLogger 2 2
let sum2 = adderWithXmlLogger -2 -2


{ "logMessage": "result=4" }
<logMessage>result=-4</logMessage


# MISU - Make Invalid States Unrepresentable
We can create custom constrained types that are more reliable than primitives like `string` or `int`. Primitives often allow so many invalid values for the purpose where they are being used that we must constantly check them and worry if the function we're calling is getting a value within the domain of values we expect. As an example, let's consider email as a `string`. An email address can certainly be stored in a `string`, but so can a great many other things. Let's create a type called `ValidEmailAddress` where instances of it can only come into existence if the email string meets the basic rules for valid emails. Functions that require valid emails can then use the supplied with confidence, eliminating many common bugs.

In [7]:

open System
open System.Net.Mail

type ValidEmailAddress = ValidEmailAddress of string

module ValidEmailAddress =

    let fromString emailStr : ValidEmailAddress =
        let (isValid, _) = MailAddress.TryCreate(emailStr)
        if not isValid then
            raise <| System.ArgumentException($"The email '{emailStr}' is not a valid email address.")
        
        ValidEmailAddress emailStr

    let toString (ValidEmailAddress(emailStr)) : string =
        emailStr


let saveValidEmail (validEmail: ValidEmailAddress) =
    printfn $"Saving {ValidEmailAddress.toString validEmail} ..."

// A bypass of the function is possible in this version
let invalidValid = ValidEmailAddress "bad email"

saveValidEmail invalidValid

// This will fail and so saveValidEmail won't ever be called. So we can't make an instance in the invalid state (MISU).
let attemptedValid = ValidEmailAddress.fromString "bad email"

saveValidEmail attemptedValid

Saving bad email ...


Error: System.ArgumentException: The email 'bad email' is not a valid email address.
   at FSI_0014.ValidEmailAddressModule.fromString(String emailStr)
   at <StartupCode$FSI_0014>.$FSI_0014.main@()

> Note: This sample code is not 100% full proof. It makes it easy and obvious how to create ValidEmailAddress instances but someone could bypass the module function and create instance of the type with   
> `let v = ValidEmailAddress "bad email"`

In [1]:
// More strict version of the module
open System
open System.Net.Mail

module ValidEmailAddress =

    type ValidEmailAddress = private ValidEmailAddress of string

    let fromString emailStr : ValidEmailAddress =
        let (isValid, _) = MailAddress.TryCreate(emailStr)
        if not isValid then
            raise <| System.ArgumentException($"The email '{emailStr}' is not a valid email address.")
        
        ValidEmailAddress emailStr

    let toString (ValidEmailAddress(emailStr)) : string =
        emailStr

// Can't bypass the fromString function like before
// let invalidValid = ValidEmailAddress "bad email"

// Create a function that relies on the ValidEmailAddress type.
let saveValidEmail (validEmail) =
    printfn $"Saving {ValidEmailAddress.toString validEmail} ..."

// This will fail and so saveValidEmail won't ever be called. So we can't make an instance in the invalid state (MISU).
let validEmail = ValidEmailAddress.fromString "bad email"

saveValidEmail validEmail

Error: System.ArgumentException: The email 'bad email' is not a valid email address.
   at FSI_0009.ValidEmailAddress.fromString(String emailStr)
   at <StartupCode$FSI_0009>.$FSI_0009.main@()